In [7]:
from nltk.corpus import wordnet
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
import spacy
import json
from pprint import pprint
nlp = spacy.load('en')
import textacy
import csv
import pandas as pd
import json
from nltk.tokenize import TreebankWordTokenizer

[nltk_data] Downloading package wordnet to /home/deshana/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/deshana/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/deshana/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
f = open("/home/deshana/Code/data/mscoco/v2_OpenEnded_mscoco_train2014_questions.json",'r')
data = json.load(f)

In [6]:
words = ['color', 'position', 'left', 'right', 'count', 'how many']

In [8]:
tokenizer = TreebankWordTokenizer()

In [9]:
chosen = []
for question in data['questions']:
    sentence = question['question']
    tokens = tokenizer.tokenize(sentence)
    sw = False
    for x in tokens:
        if x in words:
            sw = True
            break
    if sw:
        chosen.append(sentence)
    

In [13]:
chosen[0:50]

['What position is this man playing?',
 'What color is the players shirt?',
 'What color is the snow?',
 'What color is the persons headwear?',
 'What color is the toothbrush?',
 'Is this photo in color?',
 'What color is his hat?',
 'What color is the jacket?',
 'Are the walls done in a summery color?',
 'What color is the sky?',
 'What color is his coat?',
 'What companion object to the TV can be seen in the bottom right of the picture?',
 'What is the small computer on the left called?',
 'What color is the bear?',
 'What is on the ground next to the giraffe on the right?',
 'What color are the leaves?',
 'What color is the umbrella?',
 'Is the sun to the right or left of this flower?',
 'What color is the vase?',
 'What color is the napkin?',
 'What is the color of the carpet?',
 'What color is the refrigerator?',
 'What color is the plane?',
 'What color horse is closer to the camera?',
 'What is the position of the player behind the batter called?',
 "What color is the girl's shi

In [14]:
def get_triplet(sentence):
    doc = nlp(sentence)
    triplet = {}
    for chunk in doc.noun_chunks:
        #print(chunk.text, chunk.root.text, chunk.root.dep_)
              #chunk.root.head.text)
        triplet[chunk.root.dep_] = chunk.text
        print(chunk)
    return triplet

In [39]:
get_triplet("What color shirt is the man wearing?")

{'attr': 'the man', 'nsubj': 'What color shirt'}

In [15]:
def get_VP(sentence):
    pattern = r'<VERB>?<ADV>*<VERB>+'
    doc = textacy.Doc(sentence, lang='en')
    lists = textacy.extract.pos_regex_matches(doc, pattern)
    return lists

In [18]:
f = open('store_triplets.json','w')
temp = []
print(len(chosen))
for sentence in chosen[:1000]:
    triplet = {}
    triplet['sentence'] = sentence
    attrs = get_triplet(sentence)
    triplet = {**triplet, **attrs}
    VPs = get_VP(sentence)
    if (triplet is not None) and (VPs is not None):
        verbs = []
        for vp in VPs:
            verbs.append(vp.text)
        triplet['verb phrase'] = verbs
        tagged = pos_tag(word_tokenize(sentence))
        verbs = []
        temp.append(triplet)
json.dump(temp,f)
f.close()

52421


In [19]:
f = open('store_triplets.json','r')
data = json.load(f)
df_ = pd.DataFrame(data)
print(df_.head())

  ROOT               attr conj dobj        nsubj nsubjpass pobj  \
0  NaN      What position  NaN  NaN     this man       NaN  NaN   
1  NaN  the players shirt  NaN  NaN          NaN       NaN  NaN   
2  NaN           the snow  NaN  NaN          NaN       NaN  NaN   
3  NaN         What color  NaN  NaN  the persons       NaN  NaN   
4  NaN     the toothbrush  NaN  NaN          NaN       NaN  NaN   

                              sentence    verb phrase  
0   What position is this man playing?  [is, playing]  
1     What color is the players shirt?           [is]  
2              What color is the snow?           [is]  
3  What color is the persons headwear?           [is]  
4        What color is the toothbrush?           [is]  


In [20]:
attr = df_.groupby(['attr','nsubj']).count()

In [37]:
print(attr)

                                                ROOT  conj  dobj  nsubjpass  \
attr               nsubj                                                      
What               the English alphabet letter     0     0     0          0   
                   the accent color                0     0     0          0   
                   the bowl                        0     0     0          0   
                   the boxes                       0     0     0          0   
                   the circular structure          0     0     0          0   
                   the color                       0     1     1          0   
                   the darkest color               0     0     0          0   
                   the girl                        0     0     0          0   
                   the green                       0     0     0          0   
                   the guy                         0     0     0          0   
                   the item                        0

In [32]:
df_.loc[(df_['attr']=='What') & (df_['nsubj']=='the color')]

,ROOT,attr,conj,dobj,nsubj,nsubjpass,pobj,sentence,verb phrase
20,NaN,What,NaN,NaN,the color,NaN,the carpet,What is the color of the carpet?,[is]
54,NaN,What,NaN,NaN,the color,NaN,the zipper,What is the color of the zipper?,[is]
73,NaN,What,NaN,NaN,the color,NaN,NaN,What is the color of the roof shingles?,"[is, shingles]"
183,NaN,What,NaN,NaN,the color,NaN,the horse,What is the color of the horse?,[is]
203,NaN,What,NaN,NaN,the color,NaN,the smallest cow,What is the color of the smallest cow?,[is]
206,NaN,What,NaN,NaN,the color,NaN,the horses,What is the color of the horses?,[is]
208,NaN,What,NaN,NaN,the color,NaN,the bus,What is the color of the bus?,[is]
260,NaN,What,NaN,NaN,the color,NaN,the sky,What is the color of the sky?,[is]
264,NaN,What,NaN,NaN,the color,NaN,the ladies winter coat,What is the color of the ladies winter coat?,[is]
308,NaN,What,NaN,NaN,the color,NaN,the animals,What is the color of the animals?,[is]


In [36]:
print(attr)

                                                ROOT  conj  dobj  nsubjpass  \
attr               nsubj                                                      
What               the English alphabet letter     0     0     0          0   
                   the accent color                0     0     0          0   
                   the bowl                        0     0     0          0   
                   the boxes                       0     0     0          0   
                   the circular structure          0     0     0          0   
                   the color                       0     1     1          0   
                   the darkest color               0     0     0          0   
                   the girl                        0     0     0          0   
                   the green                       0     0     0          0   
                   the guy                         0     0     0          0   
                   the item                        0